<a href="https://colab.research.google.com/github/CeciliaMarson/CMLS-Homework1/blob/master/Braian_HW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1 - Computer Music (Valence and Arousal Classification)

In [0]:
import numpy as np
import librosa, librosa.display
import os
import matplotlib.pyplot as plt
import sklearn.svm
import IPython.display as ipd
import scipy as sp
import pandas as pd
import csv
import seaborn as seabornInstance
import seaborn as sns

In [0]:
!wget --no-check-certificate -r "https://drive.google.com/uc?id=12MCAyfZxquIK-dkmOX3Mi9AC2SBSJ2bG&export=download" -O "MEMD_audio.zip" 

In [0]:
!wget --no-check-certificate -r "https://drive.google.com/u/0/uc?id=10CfTAIlhD1bWER1temxNYcda7lfNupt3&export=download" -O "annotations.zip"

In [0]:
!unzip MEMD_audio.zip

In [0]:
!unzip annotations.zip

In [0]:

ipd.Audio('/content/1984.mp3') #NINE INCH NAILS!!!!!! <3

# Feature definition

### Define parameters and variables for feature computation

In [0]:
# get dataframe from annotations file
annotations = pd.read_csv('/content/annotations/annotations averaged per song/song_level/static_annotations_averaged_songs_1_2000.csv')
annotations.head()

In [0]:
from sklearn.model_selection import train_test_split
songs = '/content/'
all_songs = [f for f in os.listdir(songs) if f.endswith('.mp3')]
songs_train, songs_test = train_test_split(all_songs, test_size=0.99)

In [0]:
header = 'song rolloff zero_crossing_rate spectral_centroid spectral_bandwidth'
for i in range(1, 14):
    header += f' mfcc{i}'
header = header.split()

In [0]:
#DO NOT COMPUTE, TAKES A SHIT LOAD OF TIME, USE FEATURES - BRAIAN_NICCOLO.CSV FILE
file = open('features.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

for index, f in enumerate(all_songs):
    audio, fs = librosa.load(os.path.join(songs, f), sr=None)
    roll = librosa.feature.spectral_rolloff(audio, sr=fs, S=None, n_fft=1024, hop_length=512, freq=None, roll_percent=0.85)
    zcr =  librosa.feature.zero_crossing_rate(audio, frame_length=1024, hop_length=512, center=True)
    cent = librosa.feature.spectral_centroid(y=audio, sr=fs, S=None, n_fft=1024, hop_length=512)
    bw = librosa.feature.spectral_bandwidth(y=audio, sr=fs)
    mfcc = librosa.feature.mfcc(y=audio, sr=fs, S=None, n_mfcc=13, dct_type=2, norm='ortho')
    to_append = f'{f} {np.mean(roll)} {np.mean(zcr)} {np.mean(cent)} {np.mean(bw)}'    
    for e in mfcc:
            to_append += f' {np.mean(e)}'
    file = open('features.csv', 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())

In [0]:
features_file = pd.read_csv('/content/features - ABN.csv')
#features_file[['rolloff', 'zero_crossing_rate', 'spectral_centroid', 'spectral_bandwidth', 'onset_', 'tempo', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13']].values
#feat_array = features_file[['rolloff', 'zero_crossing_rate', 'spectral_centroid', 'spectral_bandwidth', 'onset_']].values
feat_array = np.array(features_file)

In [0]:
annot = annotations[' arousal_mean'].values

In [0]:
t_ft_max = np.max(feat_array, axis=0)
t_ft_min = np.min(feat_array, axis=0)
feat_array_norm = (feat_array - t_ft_min) / (t_ft_max - t_ft_min)
X = feat_array_norm

t_an_max = np.max(annot, axis=0)
t_an_min = np.min(annot, axis=0)
annot_norm = (annot - t_an_min) / (t_an_max - t_an_min)
y = annot_norm

X_df = pd.DataFrame(X)
X_df

In [0]:
plt.figure(figsize=(15,10))
plt.tight_layout()
seabornInstance.distplot(annot)

In [0]:
plt.plot(features_file.loc[:,'rolloff'], annotations.loc[:, ' arousal_mean'], 'o')   
plt.show()

In [0]:
#Using Pearson Correlation
database = pd.read_csv('/content/features_annotations.csv')
plt.figure(figsize=(20,12))
cor = database.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [0]:
#Correlation with output variable
cor_target = abs(cor[" valence_mean"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.5]
relevant_features

In [0]:
print(database[["rolloff","spectral_centroid"]].corr())

In [0]:
print(database[["rolloff","mfcc1"]].corr())

In [0]:
print(database[["spectral_centroid","mfcc1"]].corr())

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [0]:
from sklearn.linear_model import LinearRegression 
regressor = LinearRegression()  
regressor.fit(X_train, y_train)

In [0]:
coeff_df = pd.DataFrame(regressor.coef_, X_df.columns, columns=['Coefficient'])  
coeff_df

In [0]:
y_pred = regressor.predict(X_test)

In [0]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

df.head(15)

In [0]:
def compute_metrics(y_tested,y_predicted):
  mse = sklearn.metrics.mean_squared_error(y_tested, y_predicted)
  r2 = sklearn.metrics.r2_score(y_tested, y_predicted)
  print("Results : \n MSE = {} \n R2 = {} \n".format(mse, r2))

In [0]:
compute_metrics(y_test,y_pred)